In [ ]:
import numpy as np
from boxvectors import directions as directions
import Initial_Parameters as ip
from md import System
from md import md
from distribution import maxwellboltzmann
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.special import erfc
from scipy.constants import epsilon_0
%matplotlib inline

In [ ]:
Symbols = ip.Symbols
Coefficients = ip.Coefficients
Charges = ip.Charges
N = ip.N*np.sum(Coefficients)
L = ip.L
T = ip.T
dt = ip.dt
p_rea = ip.p_rea
std = ip.std
n_boxes_short_range = ip.n_boxes_short_range
p_error = ip.p_error
Sys= System(Symbols, Coefficients, Charges, N/2)
Labels = Sys.get_Labels()
Sigma, Epsilon = Sys.get_LJ_parameter()
r_cut_coulomb = ip.r_cut_coulomb
r_cut_LJ = ip.r_cut_LJ
r_switch = ip.r_switch
switch_parameter = ip.switch_parameter
m = Labels[:,0]

In [ ]:
def get_random_starting_Positions(N,L):
    Positions = np.zeros((N,3))
    Positions[:,0] = np.linspace(0,L[0],N, endpoint = False)
    Positions[:,1] = np.linspace(0,L[1],N, endpoint = False)
    Positions[:,2] = np.linspace(0,L[2],N, endpoint = False)
    np.random.shuffle(Positions[:,0])
    np.random.shuffle(Positions[:,1])
    np.random.shuffle(Positions[:,2])
    return Positions
Positions = get_random_starting_Positions(N,L)
Velocities = maxwellboltzmann().sample_distribution(N,m,T)
Forces = np.zeros((N,3))
# R = np.linalg.norm(Positions,axis=1)

In [ ]:
# Positions = ((directions(2).get_directions()))
# Positions = Positions +2
# Positions = np.delete(Positions, np.where(np.max(Positions, axis = 1)==4),axis = 0)
# Positions = (Positions*564e-12)+282e-12
R = np.linalg.norm(Positions,axis=1)

In [ ]:
L*=1.5

In [ ]:
MD = md(
    Positions, 
    Labels, 
    Velocities,
    Forces, 
    L, 
    T, 
    Sigma, 
    Epsilon, 
    r_switch,
    r_cut_LJ, 
    n_boxes_short_range,
    dt,
    p_rea,
    p_error,
    Symbols)

In [ ]:
MD.forces = MD.get_energy()

In [ ]:
MD.forces

In [ ]:
import os
import time
cwd = os.getcwd()
cwd

In [ ]:
start = time.time()
MD.minmimize_Energy(N_steps = 10, threshold=1e-11, Energy_save=1,  Frame_save=1, constant = dt, path=cwd)
ende = time.time()
print("\n")
print(ende-start)

In [ ]:
start = time.time()
MD.get_traj(N_steps=100, Energy_save=1, Temperature_save=1, Frame_save=1, path = cwd)
ende = time.time()
print "\n"
print ende-start

In [ ]:
MD.forces